In [1]:
import math
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [2]:
class Molecule:
    def __init__(self, file_name):

        self.D0 = 3.24
        self.r0 = 2.222
        self.S = 1.57
        self.betta = 1.4760
        self.gamma = 0.09253
        self.c = 1.13681
        self.d = 0.63397
        self.h = 0.335
        self.two_mu = 0
        self.R = 2.9
        self.D = 0.15
        self.m = 2912
        self.file_name = file_name



        with open(file_name, "r") as file:
            f_lines = file.readlines()
            while not f_lines[0].strip():
                f_lines.pop(0)
            self.atom_num = int(f_lines.pop(0))
            
            self.atom_r = {}
            line = f_lines[0].split()
    
            iter_char = iter(line)
            while True:
                try:
                    element = next(iter_char)
                    radius = next(iter_char)
                    self.atom_r[element] = float(radius)
                except StopIteration:
                    break
            f_lines.pop(0)
            
            n = 0
            self.atom_el = ['0']*self.atom_num
            self.atom_coords = [0]*self.atom_num
            for line in f_lines:
                if line.strip():
                    array = line.split()
                    self.atom_el[n] = array.pop(0)
                    self.atom_coords[n] = np.fromstring(' '.join(array), sep=' ')
                    n+=1
                else: break

    def move_coords(self, step, atom):
        new_coords = self.atom_coords[atom].copy()
        new_coords = new_coords + step
        self.atom_coords[atom] = new_coords
    
    def move_i_coord(self, step, atom, coord):
        new_coords = self.atom_coords[atom].copy()
        new_coords[coord] = new_coords[coord] + step
        self.atom_coords[atom] = new_coords
        
    def copy(self):
        new_mol = Molecule(self.file_name)
        new_mol.atom_el = self.atom_el.copy()
        new_mol.atom_coords = self.atom_coords.copy()
        new_mol.atom_num = self.atom_num
        new_mol.atom_r = self.atom_r.copy()
        return new_mol

    def box(self):
        x_max = 0
        y_max = 0
        z_max = 0
        for num in range(self.atom_num):
            atom_max_x = self.atom_coords[num][0] + self.atom_r[self.atom_el[num]]
            atom_max_y = self.atom_coords[num][1] + self.atom_r[self.atom_el[num]]
            atom_max_z = self.atom_coords[num][2] + self.atom_r[self.atom_el[num]]
            if atom_max_x > x_max:
                x_max = atom_max_x
            if atom_max_y > y_max:
                y_max = atom_max_y
            if atom_max_z > z_max:
                z_max = atom_max_z
        
        x_min = 0
        y_min = 0
        z_min = 0
        for num in range(self.atom_num):
            atom_min_x = self.atom_coords[num][0] - self.atom_r[self.atom_el[num]]
            atom_min_y = self.atom_coords[num][1] - self.atom_r[self.atom_el[num]]
            atom_min_z = self.atom_coords[num][2] - self.atom_r[self.atom_el[num]]
            if atom_min_x < x_min:
                x_min = atom_min_x
            if atom_min_y < y_min:
                y_min = atom_min_y
            if atom_min_z < z_min:
                z_min = atom_min_z

        self.x_max = x_max
        self.y_max = y_max
        self.z_max = z_max

        self.x_min = x_min
        self.y_min = y_min
        self.z_min = z_min
        self.box_volume = (self.x_max - self.x_min) * (self.y_max - self.y_min) * (self.z_max - self.z_min)
        self.box_min = min([self.x_max, self.y_max, self.z_max, self.x_min, self.y_min, self.z_min])

        return [self.x_max, self.y_max, self.z_max, self.x_min, self.y_min, self.z_min]

    def __str__(self):
        string = "{}\n".format(self.atom_num)
        for key in self.atom_r:
            string += "{} {}\n".format(key, self.atom_r[key])
        for i in range(self.atom_num):
            string += "{} {:.5f} {:.5f} {:.5f}\n".format(self.atom_el[i], self.atom_coords[i][0], self.atom_coords[i][1], self.atom_coords[i][2])
        return(string)

    def vol(self, vol, d):
        self.vol = {"value": vol, "d": d}
    
    def printToFile(self, filename):
        old_lines = ""
        with open(filename, "r") as file:
            old_lines = file.read()
        with open(filename, "w") as file:
            print(self, file=file)
            print(old_lines, file=file)
        
    
class Dot:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def random(self, x_low, x_high, y_low, y_high, z_low, z_high):
        self.x = random.uniform(x_low, x_high)
        self.y = random.uniform(y_low, y_high)
        self.z = random.uniform(z_low, z_high)

    def next(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z


In [3]:
def dotInside(dot, mol, n):
    if (
        (dot.x - mol.atom_coords[n][0]) ** 2
        + (dot.y - mol.atom_coords[n][1]) ** 2
        + (dot.z - mol.atom_coords[n][2]) ** 2
    ) <= mol.atom_r[mol.atom_el[n]] ** 2:
        return True
    return False

def volume(M):
    dot = Dot(0, 0, 0)
    box = M.box()
    dot_full = 1000
    iter_num = 100
    values = np.zeros(iter_num)
    for n in range(iter_num):
        dot_inside = 0
        for i in range(dot_full):
            dot.random(M.x_min, M.x_max, M.y_min, M.y_max, M.z_min, M.z_max)
            inside = False
            for num in range(M.atom_num):
                if dotInside(dot, M, num):
                    inside = True
                    break
            if inside:
                dot_inside += 1
        values[n] = dot_inside / dot_full * M.box_volume

    average = np.average(values)
    std = np.std(values)
    t = 1.984
    d = std * t
    print(
        "Molecule Volume = ({:.2f}+-{:.2f})A, E = {:.2f}, alpha = 0.95".format(
            average, d, d / average
        )
    )
    return average  

In [4]:
def E(mol):
    E = 0
    for j in range(mol.atom_num):
        for i in range(j):
            Eij = fc(mol, r(mol, i, j)) * (Vr(mol, r(mol, i, j)) - (b(mol, i, j) + b(mol, j, i)) / 2 * Va(mol, r(mol, i, j)))
            E += Eij
    return E

def Vr(mol, r):
    return mol.D0 / (mol.S - 1) * math.exp(-mol.betta * (2 * mol.S) ** (1/2) * (r - mol.r0))

def Va(mol, r):
    return mol.S * mol.D0 / (mol.S - 1) * math.exp(-mol.betta * (2/mol.S) ** (1/2) * (r - mol.r0))

def fc(mol, r):
    if r < mol.R - mol.D:
        return 1
    elif abs(mol.R - r) <= mol.D:
        return 1/2  - 1/2 * math.sin(math.pi * (r-mol.R) / (2 * mol.D))
    elif mol.R + mol.D < r:
        return 0

def b(mol, i, j):
    return (1 + x(mol, i, j)) ** (-1/2)
    

def x(mol, i, j):
    x = 0
    for k in range(mol.atom_num):
        if k != i and k != j:
            x_k = fc(mol, r(mol, i, k)) * g(mol, i, j, k)
            x += x_k
    return x

def g(mol, i, j, k):

    vector_ij = mol.atom_coords[j] - mol.atom_coords[i]
    vector_ik = mol.atom_coords[k] - mol.atom_coords[i]
    cos_tetta = np.dot(vector_ij, vector_ik) / (np.linalg.norm(vector_ij) * np.linalg.norm(vector_ik))
    g = mol.gamma * (1 + (mol.c / mol.d) ** 2 - mol.c ** 2 / (mol.d ** 2 + (mol.h + cos_tetta) ** 2))
    return g

def r(mol, i, j):
   vector_ij = mol.atom_coords[j] - mol.atom_coords[i]
   return np.linalg.norm(vector_ij)

In [32]:
def grad(mol, atom):
    delta = 0.001
    grad = np.zeros(3)
    mol_up = mol.copy()
    mol_down = mol.copy()
    for i in range(3):
        mol_up.move_i_coord(delta, atom, i)
        mol_down.move_i_coord(-delta, atom, i)
        grad[i] = (E(mol_up) - E(mol_down)) / (2 * delta)
        mol_up = mol.copy()
        mol_down = mol.copy()
    return grad

def F(mol, atom):
    F = - grad(mol, atom)
    return F

def fullF(mol):
    f = np.zeros((mol.atom_num, 3))
    for atom in range(mol.atom_num):
        f[atom] = F(mol, atom)
    return f

def relaxate(mol):
    low_f = 0.01
    sum_f = 1
    f = np.zeros((mol.atom_num, 3))
    n = 0
    while sum_f - low_f > 0:
        sum_f = 0
        for i in range(mol.atom_num):
            f[i] = F(mol, i)
            sum_f += np.linalg.norm(f[i])
        mol.atom_coords += 0.01 * f
        n += 1
        if n%20 == 0:
            print(n, sum_f)
            mol.printToFile("DataEx6relaxate.xyz")
    print('E = ', E(mol))
    return f

In [40]:
def velocity(T, n, m):
    phi = random.uniform(0, 2 * np.pi)
    r = random.uniform(0.0000001, 1)

    ro = np.sqrt(-2*np.log(r))
    x =  ro * np.cos(6.28 * phi)
    
    k = 0.00008617
    mu = 0
    sigma = np.sqrt(2 * k * T / m)
    
    y = mu + sigma * x
    return y

def find_velocities(mol, T, m):
    velocities = np.zeros((mol.atom_num, 3))
    for atom in range(mol.atom_num):
        for i in range(3):
            velocities[atom][i] = velocity(T, mol.atom_num, m)
    return velocities

def volumeT(molecule, T):
    m = molecule.m
    velocities = find_velocities(molecule, T, m)
    dtime = 1
    vf = f = relaxate(molecule)
    volumes = np.zeros(50)
    mol = molecule.copy()

    for t in range(10000):
        mol.atom_coords += velocities * dtime + 0.5 * f * dtime ** 2 / m
        vf = f

        f= fullF(mol)
        velocities += 0.5 * dtime * (vf + f) / m

        if t%200 == 0:
            volumes[int(t/200)-1] = volume(mol)
    
    return np.mean(volumes)

def thermalTest(molecule, T): 
    m = molecule.m
    velocities = find_velocities(molecule, T, m)
    dtime = 1
    f = np.zeros([10, 3])
    vf = f
    mol = molecule.copy()

    for t in range(10000):
        mol.atom_coords += velocities * dtime + 0.5 * f * dtime ** 2 / m
        vf = f

        f = fullF(mol)
        velocities += 0.5 * dtime * (vf + f) / m

        if t%500 == 0:
            print(t, end = " ")

    return mol

def checkEquality(mol_base, mol):
    relaxate(mol)
    if np.abs(E(mol_base) - E(mol)) < 0.001:
            return "Molecule is ok"
    return "Molecule has been desintegrated"

def thermalResistance(mol, temp):
    mol_base = mol.copy()
    mol = thermalTest(mol, temp)
    mol.printToFile("DataEx6.xyz")
    print(checkEquality(mol_base, mol))
    print(mol)
    return mol

In [33]:
mol = Molecule("DataEx5Opt.xyz")
mol1 = mol.copy()
mol1 = thermalResistance(mol1, 900)